# 01 load libraries

In [1]:
import nitime
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# Import the time-series objects:
from nitime.timeseries import TimeSeries
from scipy import stats 
# Import the analysis objects:
from nitime.analysis import SpectralAnalyzer, FilterAnalyzer, NormalizationAnalyzer

import torch
import torch.nn as nn
import torch.nn.functional as F

from scipy.optimize import curve_fit

/tmp/ipykernel_4138299/1521739541.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# 02 draw!

In [ ]:
plt.figure(figsize=(20,5))

# 2-1 ABCD
plt.subplot(121)
data_dir = '/storage/bigdata/ABCD/fmriprep/1.rs_fmri/5.ROI_DATA'
sub_list = os.listdir(data_dir)

sub = sub_list[0]
file_dir = os.path.join(data_dir, sub)
file_name = os.path.join(file_dir, 'hcp_mmp1_'+sub+'.npy')

y = np.load(file_name)[20:20+348].T

TR = 0.8

sample_whole = np.zeros(348,)

for i in range(180):
    sample_whole+=y[i]

sample_whole /= 180   

T = TimeSeries(sample_whole, sampling_interval=TR)
S_original = SpectralAnalyzer(T)

# Lorentzian function fitting
xdata = np.array(S_original.spectrum_fourier[0][1:])
ydata = np.abs(S_original.spectrum_fourier[1][1:])

def lorentzian_function(x, s0, corner):
    return (s0*corner**2) / (x**2 + corner**2)

# 초기 매개변수 설정
p0 = [0, 0.006]

# 로런치안 함수 피팅
popt, pcov = curve_fit(lorentzian_function, xdata, ydata, p0=p0, maxfev = 5000)

f1 = popt[1]

knee = round(popt[1]/(1/(sample_whole.shape[0]*TR)))

if knee <= 0:
    knee = 1

def modified_lorentzian_function(x, beta_low, beta_high, A, B, corner):
    return np.where(x < corner, A * x**beta_low, B * x**beta_high)
    #return A*x**(-beta_low) / (1+(x/corner)**beta_high)

p1 = [2, 1, 23, 25, 0.16]

popt_mo, pcov = curve_fit(modified_lorentzian_function, xdata[knee:], ydata[knee:], p0=p1, maxfev = 50000)
pink = round(popt_mo[-1]/(1/(sample_whole.shape[0]*TR)))
f2 = popt_mo[-1]

plt.loglog(xdata, ydata, label = 'original PSD')


# ultralow : 1 ~ knee+1
x_u = np.log10(S_original.spectrum_fourier[0][1:knee+1])
y_u = np.log10(np.abs(S_original.spectrum_fourier[1])[1:knee+1])
z_u = np.polyfit(x_u, y_u, 1) # (X,Y,차원) 정의
p_u = np.poly1d(z_u) # 1차원 다항식에 대한 연산을 캡슐화

plt.plot(10**x_u, 10**(p_u(x_u))*1.5, color = 'yellowgreen', label='fitted line for ultralow frequency')
print("fitted line for ultralow frequency is y=%.6fx+(%.6f)"%(z_u[0],z_u[1]))

plt.axvline(x=S_original.spectrum_fourier[0][knee], linestyle='--', color='r')

# low : knee ~ pink+1
x_l = np.log10(S_original.spectrum_fourier[0][knee:pink+1])
y_l = np.log10(np.abs(S_original.spectrum_fourier[1])[knee:pink+1])
z_l = np.polyfit(x_l, y_l, 1) # (X,Y,차원) 정의
p_l = np.poly1d(z_l) # 1차원 다항식에 대한 연산을 캡슐화

plt.plot(10**x_l, 10**(p_l(x_l))*1.4, color = 'orange', label='fitted line for low frequency')
print("fitted line for low frequency is y=%.6fx+(%.6f)"%(z_l[0],z_l[1]))

plt.axvline(x=S_original.spectrum_fourier[0][pink], linestyle='--', color='g')


# high : pink ~
x_h = np.log10(S_original.spectrum_fourier[0][pink:])
y_h = np.log10(np.abs(S_original.spectrum_fourier[1])[pink:])
z_h = np.polyfit(x_h, y_h, 1) # (X,Y,차원) 정의
p_h = np.poly1d(z_h) # 1차원 다항식에 대한 연산을 캡슐화

plt.plot(10**x_h, 10**(p_h(x_h))*1.3, color = 'hotpink', label='fitted line for high frequency')
print("fitted line for high frequency is y=%.6fx+(%.6f)"%(z_h[0],z_h[1]))


plt.xlabel('log(frequency)')

plt.ylabel('log(power)')


# 2-2 UKB
plt.subplot(122)
data_dir = '/scratch/connectome/stellasybae/UKB_ROI'
sub_list = os.listdir(data_dir)

sub = sub_list[1]
file_dir = os.path.join(data_dir, sub)
file_name = os.path.join(file_dir, 'hcp_mmp1_'+sub+'.npy')

y = np.load(file_name)[20:20+464].T

TR = 0.735

sample_whole = np.zeros(464,)

for i in range(180):
    sample_whole+=y[i]

sample_whole /= 180   

T = TimeSeries(sample_whole, sampling_interval=TR)
S_original = SpectralAnalyzer(T)

# Lorentzian function fitting
xdata = np.array(S_original.spectrum_fourier[0][1:])
ydata = np.abs(S_original.spectrum_fourier[1][1:])


def lorentzian_function(x, s0, corner):
    return (s0*corner**2) / (x**2 + corner**2)

# 초기 매개변수 설정
p0 = [0, 0.006]

# 로런치안 함수 피팅
popt, pcov = curve_fit(lorentzian_function, xdata, ydata, p0=p0, maxfev = 5000)

f1 = popt[1]

knee = round(popt[1]/(1/(sample_whole.shape[0]*TR)))

if knee <= 0:
    knee = 1

def modified_lorentzian_function(x, beta_low, beta_high, A, B, corner):
    return np.where(x < corner, A * x**beta_low, B * x**beta_high)
    #return A*x**(-beta_low) / (1+(x/corner)**beta_high)

p1 = [2, 1, 23, 25, 0.16]

popt_mo, pcov = curve_fit(modified_lorentzian_function, xdata[knee:], ydata[knee:], p0=p1, maxfev = 50000)
pink = round(popt_mo[-1]/(1/(sample_whole.shape[0]*TR)))
f2 = popt_mo[-1]

plt.loglog(xdata, ydata, label = 'original PSD')


# ultralow : 1 ~ knee+1
x_u = np.log10(S_original.spectrum_fourier[0][1:knee+1])
y_u = np.log10(np.abs(S_original.spectrum_fourier[1])[1:knee+1])
z_u = np.polyfit(x_u, y_u, 1) # (X,Y,차원) 정의
p_u = np.poly1d(z_u) # 1차원 다항식에 대한 연산을 캡슐화

plt.plot(10**x_u, 10**(p_u(x_u))*1.5, color = 'yellowgreen', label='fitted line for ultralow frequency')
print("fitted line for ultralow frequency is y=%.6fx+(%.6f)"%(z_u[0],z_u[1]))

plt.axvline(x=S_original.spectrum_fourier[0][knee], linestyle='--', color='r')

# low : knee ~ pink+1
x_l = np.log10(S_original.spectrum_fourier[0][knee:pink+1])
y_l = np.log10(np.abs(S_original.spectrum_fourier[1])[knee:pink+1])
z_l = np.polyfit(x_l, y_l, 1) # (X,Y,차원) 정의
p_l = np.poly1d(z_l) # 1차원 다항식에 대한 연산을 캡슐화

plt.plot(10**x_l, 10**(p_l(x_l))*1.4, color = 'orange', label='fitted line for low frequency')
print("fitted line for low frequency is y=%.6fx+(%.6f)"%(z_l[0],z_l[1]))

plt.axvline(x=S_original.spectrum_fourier[0][pink], linestyle='--', color='g')


# high : pink ~
x_h = np.log10(S_original.spectrum_fourier[0][pink:])
y_h = np.log10(np.abs(S_original.spectrum_fourier[1])[pink:])
z_h = np.polyfit(x_h, y_h, 1) # (X,Y,차원) 정의
p_h = np.poly1d(z_h) # 1차원 다항식에 대한 연산을 캡슐화

plt.plot(10**x_h, 10**(p_h(x_h))*1.3, color = 'hotpink', label='fitted line for high frequency')
print("fitted line for high frequency is y=%.6fx+(%.6f)"%(z_h[0],z_h[1]))


plt.xlabel('log(frequency)')

plt.ylabel('log(power)')

plt.legend(loc='lower left', bbox_to_anchor=(1.0,0.5))

plt.tight_layout()
plt.savefig('frequency_dividing.png', 
    dpi=300, 
    facecolor='w', 
    edgecolor='w',
    orientation='portrait', 
    format=None,
    transparent=False, 
    bbox_inches='tight', 
    pad_inches=0.1,
    metadata=None)

plt.show()